 1. Load dataset


In [7]:
import pandas as pd
import numpy as np
import os
dataset_path = "netflix_titles.csv"
df = pd.read_csv(dataset_path)

print("Original Shape:", df.shape)
print("Memory usage: {:.2f} MB".format(df.memory_usage(deep=True).sum() / 1024**2))
print(df.head())



Original Shape: (8807, 12)
Memory usage: 7.79 MB
  show_id     type                  title         director  \
0      s1    Movie   Dick Johnson Is Dead  Kirsten Johnson   
1      s2  TV Show          Blood & Water              NaN   
2      s3  TV Show              Ganglands  Julien Leclercq   
3      s4  TV Show  Jailbirds New Orleans              NaN   
4      s5  TV Show           Kota Factory              NaN   

                                                cast        country  \
0                                                NaN  United States   
1  Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...   South Africa   
2  Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...            NaN   
3                                                NaN            NaN   
4  Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...          India   

           date_added  release_year rating   duration  \
0  September 25, 2021          2020  PG-13     90 min   
1  September 24, 2021          2021  

2. Assess missing and duplicates

In [8]:

missing_df = df.isnull().sum().reset_index()
missing_df.columns = ["column", "missing_count"]
missing_df["missing_percent"] = (missing_df["missing_count"] / len(df)) * 100

print("\nMissing values before cleaning:\n", missing_df)
print("\nDuplicate rows:", df.duplicated().sum())



Missing values before cleaning:
           column  missing_count  missing_percent
0        show_id              0         0.000000
1           type              0         0.000000
2          title              0         0.000000
3       director           2634        29.908028
4           cast            825         9.367549
5        country            831         9.435676
6     date_added             10         0.113546
7   release_year              0         0.000000
8         rating              4         0.045418
9       duration              3         0.034064
10     listed_in              0         0.000000
11   description              0         0.000000

Duplicate rows: 0


 3. Cleaning process

In [9]:
df_cleaned = df.copy()

# --- Handle missing values
critical_cols = ['show_id', 'type', 'title']
df_cleaned.dropna(subset=critical_cols, inplace=True)   # drop rows missing critical IDs
fill_cols = ['director', 'cast', 'country', 'listed_in', 'description']
for col in fill_cols:
    df_cleaned[col] = df_cleaned[col].fillna("Unknown")

if "rating" in df_cleaned.columns:
    df_cleaned["rating"] = df_cleaned["rating"].fillna(df_cleaned["rating"].mode()[0])

# --- Remove duplicates
df_cleaned.drop_duplicates(inplace=True)

# --- Standardize text
df_cleaned["type"] = df_cleaned["type"].str.strip().str.title()
df_cleaned["type"] = df_cleaned["type"].replace({"Tv Show": "TV Show"})

df_cleaned["country"] = df_cleaned["country"].str.strip()
df_cleaned["country"] = df_cleaned["country"].apply(lambda x: x.split(",")[0] if isinstance(x, str) else x)
country_map = {"USA": "United States", "UK": "United Kingdom", "South Korea": "Korea"}
df_cleaned["country"] = df_cleaned["country"].replace(country_map)

df_cleaned["rating"] = df_cleaned["rating"].str.upper().str.strip()

# --- Standardize dates
df_cleaned["date_added"] = pd.to_datetime(df_cleaned["date_added"], errors="coerce")

# --- Validate numeric columns
df_cleaned["release_year"] = pd.to_numeric(df_cleaned["release_year"], errors="coerce")
print("\nRelease year range:", df_cleaned["release_year"].min(), "→", df_cleaned["release_year"].max())

# --- Clean column names
df_cleaned.columns = (
    df_cleaned.columns.str.lower()
    .str.strip()
    .str.replace(" ", "_")
    .str.replace("-", "_")
)



Release year range: 1925 → 2021


4. Final validation

In [10]:
print("\nFinal Shape:", df_cleaned.shape)
print("Remaining missing values:\n", df_cleaned.isnull().sum())


Final Shape: (8807, 12)
Remaining missing values:
 show_id          0
type             0
title            0
director         0
cast             0
country          0
date_added      98
release_year     0
rating           0
duration         3
listed_in        0
description      0
dtype: int64


5. Save cleaned data & report

In [11]:
output_path = "netflix_titles_cleaned.csv"
report_path = "netflix_data_cleaning_report.txt"

df_cleaned.to_csv(output_path, index=False)

with open(report_path, "w") as f:
    f.write("Netflix Data Cleaning Report\n")
    f.write("="*40 + "\n")
    f.write(f"Original shape: {df.shape}\n")
    f.write(f"Final shape: {df_cleaned.shape}\n")
    f.write(f"Total missing before: {missing_df['missing_count'].sum()}\n")
    f.write(f"Total missing after: {df_cleaned.isnull().sum().sum()}\n")
    f.write(f"Duplicates removed: {df.duplicated().sum()}\n")
    f.write(f"Release year range: {df_cleaned['release_year'].min()} → {df_cleaned['release_year'].max()}\n")
    f.write("Rating mode used to fill nulls: " + df_cleaned["rating"].mode()[0] + "\n")

print(f"\nCleaned dataset saved as: {output_path}")
print(f"Cleaning report saved as: {report_path}")


Cleaned dataset saved as: netflix_titles_cleaned.csv
Cleaning report saved as: netflix_data_cleaning_report.txt
